In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')
datadir = '/content/gdrive/My Drive/colab/TimeSeries/dataset/7'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
import torch
import torch.nn as nn
import numpy as np
import time
import math
from matplotlib import pyplot
import pandas as pd

torch.manual_seed(0)
np.random.seed(0)

# S is the source sequence length
# T is the target sequence length
# N is the batch size
# E is the feature number

#src = torch.rand((10, 32, 512)) # (S,N,E) 
#tgt = torch.rand((20, 32, 512)) # (T,N,E)
#out = transformer_model(src, tgt)
#
#print(out)

input_window = 100
output_window = 5 # multistep prediction      # 10
batch_size = 10 # batch size                  # 3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()       
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        #pe.requires_grad = False
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:x.size(0), :]
       

class TransAm(nn.Module):
    def __init__(self,feature_size=250,num_layers=1,dropout=0.1):
        super(TransAm, self).__init__()
        self.model_type = 'Transformer'
        
        self.src_mask = None
        self.pos_encoder = PositionalEncoding(feature_size)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=feature_size, nhead=10, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)        
        self.decoder = nn.Linear(feature_size,1)
        self.init_weights()

    def init_weights(self):
        initrange = 0.1    
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self,src):
        if self.src_mask is None or self.src_mask.size(0) != len(src):
            device = src.device
            mask = self._generate_square_subsequent_mask(len(src)).to(device)
            self.src_mask = mask

        src = self.pos_encoder(src)
        output = self.transformer_encoder(src,self.src_mask)#, self.src_mask)
        output = self.decoder(output)
        return output

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask



# if window is 100 and prediction step is 1
# in -> [0..99]
# target -> [1..100]
def create_inout_sequences(input_data, tw):
    inout_seq = []
    L = len(input_data)
    for i in range(L-tw):
        train_seq = np.append(input_data[i:i+tw][:-output_window] , output_window * [0])
        train_label = input_data[i:i+tw]
        #train_label = input_data[i+output_window:i+tw+output_window]
        inout_seq.append((train_seq ,train_label))
    return torch.FloatTensor(inout_seq)

def get_data():

    #time        = np.arange(0, 400, 0.1)
    #amplitude   = np.sin(time) + np.sin(time*0.05) +np.sin(time*0.12) *np.random.normal(-0.2, 0.2, len(time))
    
    #from pandas import read_csv
    series = pd.read_csv(datadir + '/daily-min-temperatures.csv', header=0, index_col=0, parse_dates=True, squeeze=True)
    
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler(feature_range=(-1, 1)) 
    amplitude = series.to_numpy()
    amplitude = scaler.fit_transform(amplitude.reshape(-1, 1)).reshape(-1)
    
    
    sampels = 2800
    train_data = amplitude[:sampels]
    test_data = amplitude[sampels:]

    # convert our train data into a pytorch train tensor
    #train_tensor = torch.FloatTensor(train_data).view(-1)
    # todo: add comment.. 
    train_sequence = create_inout_sequences(train_data,input_window)
    train_sequence = train_sequence[:-output_window] #todo: fix hack?

    #test_data = torch.FloatTensor(test_data).view(-1) 
    test_data = create_inout_sequences(test_data,input_window)
    test_data = train_sequence[:-output_window] #todo: fix hack?

    return train_sequence.to(device),test_data.to(device)

def get_batch(source, i,batch_size):
    seq_len = min(batch_size, len(source) - 1 - i)
    data = source[i:i+seq_len]    
    input = torch.stack(torch.stack([item[0] for item in data]).chunk(input_window,1)) # 1 is feature size
    target = torch.stack(torch.stack([item[1] for item in data]).chunk(input_window,1))
    return input, target


def train(train_data):
    model.train() # Turn on the train mode
    total_loss = 0.
    start_time = time.time()

    for batch, i in enumerate(range(0, len(train_data) - 1, batch_size)):
        data, targets = get_batch(train_data, i,batch_size)
        optimizer.zero_grad()
        output = model(data)        
        loss = criterion(output[-output_window:], targets[-output_window:])
        #loss = criterion(output, targets)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        log_interval = int(len(train_data) / batch_size / 5)
        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:02.6f} | {:5.2f} ms | '
                  'loss {:5.5f} | ppl {:8.2f}'.format(
                    epoch, batch, len(train_data) // batch_size, scheduler.get_lr()[0],
                    elapsed * 1000 / log_interval,
                    cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()

def plot_and_loss(eval_model, data_source,epoch):
    eval_model.eval() 
    total_loss = 0.
    test_result = torch.Tensor(0)    
    truth = torch.Tensor(0)
    with torch.no_grad():
        for i in range(0, len(data_source) - 1):
            data, target = get_batch(data_source, i,1)
            # look like the model returns static values for the output window
            output = eval_model(data)                    
            total_loss += criterion(output[-output_window:], target[-output_window:]).item()
            #total_loss += criterion(output, target).item()
            test_result = torch.cat((test_result, output[-1].view(-1).cpu()), 0) #todo: check this. -> looks good to me
            truth = torch.cat((truth, target[-1].view(-1).cpu()), 0)
            
    #test_result = test_result.cpu().numpy()
    len(test_result)

    pyplot.plot(test_result,color="red")
    pyplot.plot(truth[:500],color="blue")
    pyplot.plot(test_result-truth,color="green")
    pyplot.grid(True, which='both')
    pyplot.axhline(y=0, color='k')
    pyplot.savefig(datadir + '/graph/transformer-epoch%d.png'%epoch)
    pyplot.close()
    
    return total_loss / i


def predict_future(eval_model, data_source,steps):
    eval_model.eval() 
    total_loss = 0.
    test_result = torch.Tensor(0)    
    truth = torch.Tensor(0)
    _ , data = get_batch(data_source, 0,1)
    with torch.no_grad():
        for i in range(0, steps,1):
            input = torch.clone(data[-input_window:])
            input[-output_window:] = 0     
            output = eval_model(data[-input_window:])                        
            data = torch.cat((data, output[-1:]))
            
    data = data.cpu().view(-1)
    

    pyplot.plot(data,color="red")       
    pyplot.plot(data[:input_window],color="blue")
    pyplot.grid(True, which='both')
    pyplot.axhline(y=0, color='k')
    pyplot.savefig(datadir + '/transformer-multi-future%d.png'%steps)
    pyplot.close()
        
# entweder ist hier ein fehler im loss oder in der train methode, aber die ergebnisse sind unterschiedlich 
# auch zu denen der predict_future
def evaluate(eval_model, data_source):
    eval_model.eval() # Turn on the evaluation mode
    total_loss = 0.
    eval_batch_size = 1000
    with torch.no_grad():
        for i in range(0, len(data_source) - 1, eval_batch_size):
            data, targets = get_batch(data_source, i,eval_batch_size)
            output = eval_model(data)            
            total_loss += len(data[0])* criterion(output[-output_window:], targets[-output_window:]).cpu().item()
            # total_loss += len(data[0])* criterion(output, targets).cpu().item()
    return total_loss / i

In [ ]:
train_data, val_data = get_data()
model = TransAm().to(device)

criterion = nn.MSELoss()
lr = 0.005 
#optimizer = torch.optim.SGD(model.parameters(), lr=lr)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.98)

best_val_loss = float("inf")
epochs = 100 # The number of epochs
best_model = None

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train(train_data)
    
    
    if(epoch % 10 is 0):
        val_loss = plot_and_loss(model, val_data,epoch)
        predict_future(model, val_data,200)
    else:
        val_loss = evaluate(model, val_data)
        
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.5f} | valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                     val_loss, math.exp(val_loss)))
    print('-' * 89)

    #if val_loss < best_val_loss:
    #    best_val_loss = val_loss
    #    best_model = model

    scheduler.step() 

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:351: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)


| epoch   1 |    53/  269 batches | lr 0.005000 | 23.83 ms | loss 5.31915 | ppl   204.21
| epoch   1 |   106/  269 batches | lr 0.005000 | 14.21 ms | loss 0.08926 | ppl     1.09
| epoch   1 |   159/  269 batches | lr 0.005000 | 14.01 ms | loss 0.09312 | ppl     1.10
| epoch   1 |   212/  269 batches | lr 0.005000 | 14.00 ms | loss 0.05412 | ppl     1.06
| epoch   1 |   265/  269 batches | lr 0.005000 | 14.01 ms | loss 0.04681 | ppl     1.05
-----------------------------------------------------------------------------------------
| end of epoch   1 | time:  4.89s | valid loss 0.14884 | valid ppl     1.16
-----------------------------------------------------------------------------------------
| epoch   2 |    53/  269 batches | lr 0.004802 | 14.30 ms | loss 0.06472 | ppl     1.07
| epoch   2 |   106/  269 batches | lr 0.004802 | 14.26 ms | loss 0.07996 | ppl     1.08
| epoch   2 |   159/  269 batches | lr 0.004802 | 14.19 ms | loss 0.08112 | ppl     1.08
| epoch   2 |   212/  269 batche

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:351: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)


| epoch  11 |    53/  269 batches | lr 0.004004 | 15.05 ms | loss 0.12755 | ppl     1.14
| epoch  11 |   106/  269 batches | lr 0.004004 | 13.93 ms | loss 0.12658 | ppl     1.13
| epoch  11 |   159/  269 batches | lr 0.004004 | 13.62 ms | loss 0.10638 | ppl     1.11
| epoch  11 |   212/  269 batches | lr 0.004004 | 14.11 ms | loss 0.10512 | ppl     1.11
| epoch  11 |   265/  269 batches | lr 0.004004 | 14.04 ms | loss 0.11381 | ppl     1.12
-----------------------------------------------------------------------------------------
| end of epoch  11 | time:  4.40s | valid loss 0.12312 | valid ppl     1.13
-----------------------------------------------------------------------------------------
| epoch  12 |    53/  269 batches | lr 0.003924 | 14.44 ms | loss 0.11934 | ppl     1.13
| epoch  12 |   106/  269 batches | lr 0.003924 | 14.00 ms | loss 0.12429 | ppl     1.13
| epoch  12 |   159/  269 batches | lr 0.003924 | 13.76 ms | loss 0.10939 | ppl     1.12
| epoch  12 |   212/  269 batche

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:351: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)


...

| epoch  71 |    53/  269 batches | lr 0.001191 | 15.05 ms | loss 0.12224 | ppl     1.13
| epoch  71 |   106/  269 batches | lr 0.001191 | 13.85 ms | loss 0.10391 | ppl     1.11
| epoch  71 |   159/  269 batches | lr 0.001191 | 14.18 ms | loss 0.09789 | ppl     1.10
| epoch  71 |   212/  269 batches | lr 0.001191 | 13.99 ms | loss 0.07416 | ppl     1.08
| epoch  71 |   265/  269 batches | lr 0.001191 | 13.88 ms | loss 0.08806 | ppl     1.09
-----------------------------------------------------------------------------------------
| end of epoch  71 | time:  4.41s | valid loss 0.12338 | valid ppl     1.13
-----------------------------------------------------------------------------------------
| epoch  72 |    53/  269 batches | lr 0.001167 | 14.57 ms | loss 0.11855 | ppl     1.13
| epoch  72 |   106/  269 batches | lr 0.001167 | 13.89 ms | loss 0.10168 | ppl     1.11
| epoch  72 |   159/  269 batches | lr 0.001167 | 14.22 ms | loss 0.09682 | ppl     1.10
| epoch  72 |   212/  269 batche

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:351: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)


| epoch  81 |    53/  269 batches | lr 0.000973 | 15.59 ms | loss 0.11799 | ppl     1.13
| epoch  81 |   106/  269 batches | lr 0.000973 | 14.06 ms | loss 0.10248 | ppl     1.11
| epoch  81 |   159/  269 batches | lr 0.000973 | 13.98 ms | loss 0.09673 | ppl     1.10
| epoch  81 |   212/  269 batches | lr 0.000973 | 14.05 ms | loss 0.07442 | ppl     1.08
| epoch  81 |   265/  269 batches | lr 0.000973 | 13.93 ms | loss 0.08805 | ppl     1.09
-----------------------------------------------------------------------------------------
| end of epoch  81 | time:  4.45s | valid loss 0.12314 | valid ppl     1.13
-----------------------------------------------------------------------------------------
| epoch  82 |    53/  269 batches | lr 0.000954 | 14.45 ms | loss 0.11958 | ppl     1.13
| epoch  82 |   106/  269 batches | lr 0.000954 | 13.80 ms | loss 0.10339 | ppl     1.11
| epoch  82 |   159/  269 batches | lr 0.000954 | 14.04 ms | loss 0.09677 | ppl     1.10
| epoch  82 |   212/  269 batche

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:351: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)


| epoch  91 |    53/  269 batches | lr 0.000795 | 15.35 ms | loss 0.11720 | ppl     1.12
| epoch  91 |   106/  269 batches | lr 0.000795 | 14.05 ms | loss 0.10103 | ppl     1.11
| epoch  91 |   159/  269 batches | lr 0.000795 | 13.95 ms | loss 0.09610 | ppl     1.10
| epoch  91 |   212/  269 batches | lr 0.000795 | 14.19 ms | loss 0.07341 | ppl     1.08
| epoch  91 |   265/  269 batches | lr 0.000795 | 13.92 ms | loss 0.08749 | ppl     1.09
-----------------------------------------------------------------------------------------
| end of epoch  91 | time:  4.44s | valid loss 0.12308 | valid ppl     1.13
-----------------------------------------------------------------------------------------
| epoch  92 |    53/  269 batches | lr 0.000779 | 14.89 ms | loss 0.11711 | ppl     1.12
| epoch  92 |   106/  269 batches | lr 0.000779 | 13.92 ms | loss 0.09907 | ppl     1.10
| epoch  92 |   159/  269 batches | lr 0.000779 | 13.71 ms | loss 0.09570 | ppl     1.10
| epoch  92 |   212/  269 batche